In [50]:
import pandas as pd
import os
import numpy as np

csv_files = [f for f in os.listdir('../data/raw') if f.endswith('.csv')]

In [51]:
df_dirty = pd.read_csv(f'../data/raw/{csv_files[0]}')

In [52]:
cols_to_clean = ['se_title', 'se_description', 'n3k_title', 'n3k_body', 'bs_title', 'bs_body']

In [53]:
df_dirty[df_dirty['n3k_body'].str.len() < 50]['n3k_body']

Series([], Name: n3k_body, dtype: object)

In [55]:
# Replace bodies with NA
df_dirty[((df_dirty['n3k_body'].str.contains('cookies', na=False))
         | (df_dirty['n3k_body'].str.contains('JavaScript', na=False)))
         & (df_dirty['n3k_body'].str.len() < 400)]['n3k_body']

# Replace titles with NA
df_dirty.loc[(df_dirty['n3k_title'].str.contains('robot', na=False))
         |(df_dirty['n3k_title'].str.contains('Subscribe', na=False))
         |(df_dirty['n3k_title'].str.contains('Register', na=False))
         &(df_dirty['n3k_title'].str.len() < 20), 'n3k_title']

7      Subscribe to read
13     Subscribe to read
19     Subscribe to read
32      Are you a robot?
42      Are you a robot?
65      Register to read
77     Subscribe to read
101    Subscribe to read
108     Are you a robot?
120     Are you a robot?
261     Are you a robot?
263     Are you a robot?
279     Are you a robot?
284    Subscribe to read
288     Are you a robot?
290     Are you a robot?
298     Are you a robot?
303     Are you a robot?
305     Are you a robot?
321     Are you a robot?
324    Subscribe to read
361    Subscribe to read
394    Subscribe to read
395     Are you a robot?
404     Are you a robot?
434     Are you a robot?
448     Are you a robot?
475     Are you a robot?
478     Are you a robot?
479     Are you a robot?
490     Are you a robot?
493     Are you a robot?
Name: n3k_title, dtype: object

In [15]:

df_dirty[cols_to_clean].apply(lambda x: x.replace('UBS', 'HIIIIIIII'))

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
# Eventually create loop that cleans all data in raw or recognizes files that are not cleaned yet
for csv_file in csv_files:
    pd.read_csv(f'../data/raw/{csv_file}')